### Push local data to Staging S3 location

- Put locally data to S3 staging location
- Data can be of any format (say json, csv, sas7bdat, etc) and can be from any location. Place a copy of this raw data in staging S3 location

In [99]:
""" 
    Read properties using configparser
"""
import configparser
import os

config = configparser.ConfigParser()
config.read('config.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS_ACCESS_KEYS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_ACCESS_KEYS']['AWS_SECRET_ACCESS_KEY']

local_i94_dir = config['LOCAL_DATA_PATH']['I94_DATA_DIR']
local_iso_location_file = config['LOCAL_DATA_PATH']['ISO_LOC_FILE']
local_us_demography_file = config['LOCAL_DATA_PATH']['US_DEMO_FILE']

In [71]:
import boto3

from datetime import date

In [95]:
import boto3

s3 = boto3.client('s3')
#s3 = boto3.resource("s3")

s3_bucket=config['S3']['S3_BUCKET']
s3_base_staging_dir=config['S3']['S3_BASE_STAGING_DIR']
s3_today_load_dir="date="+str(date.today())

# create new directoty inside staging for current day
#s3.put_object(Bucket=s3_bucket, Key=("{}/{}/".format(s3_base_staging_dir, s3_today_load_dir)))

s3_staging_dir = "s3://{}/{}/{}".format(s3_bucket, s3_base_staging_dir, s3_today_load_dir)
print(s3_staging_dir)

s3://dend-project-us-immigration/staging/date=2019-09-20


In [83]:
bucketName = s3_bucket
Key = "data/IP2LOCATION_ISO3166.csv"
outPutname = "{}/{}/{}".format(s3_base_staging_dir, s3_today_load_dir, "ISO_location_code_lookup.csv")

s3.upload_file(Key, bucketName, outPutname)

In [87]:
# Upload ISO Location File

uploadFile = local_iso_location_file
outPutname = "{}/{}/{}".format(s3_base_staging_dir, s3_today_load_dir, "ISO_location_code_lookup.csv")

#s3.upload_file(uploadFile, s3_bucket, outPutname)

In [96]:
# Upload US demography File

uploadFile = local_us_demography_file
outPutname = "{}/{}/{}".format(s3_base_staging_dir, s3_today_load_dir, "US_Demographics.csv")

s3.upload_file(uploadFile, s3_bucket, outPutname)

In [105]:
# Upload I94 files (in SAS format)


for filename in os.listdir(local_i94_dir):
    if filename.endswith(".sas7bdat"): 
        uploadFile = os.path.join(local_i94_dir, filename)
        outPutname = "{}/{}".format(s3_base_staging_dir, s3_today_load_dir + "/i94_data/")
        #s3.upload_file(uploadFile, s3_bucket, outPutname)
        continue
    else:
        continue

In [93]:
s3 = boto3.resource("s3")
capstone_bucket = s3.Bucket(s3_bucket)

for obj in capstone_bucket.objects.filter(Prefix='staging'):
    print(obj.key)

staging/
staging/date=2019-09-20/
staging/date=2019-09-20/ISO_location_code_lookup.csv
staging/date=2019-09-20/US_Demographics.csv
